```
Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you
may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
implied. See the License for the specific language governing
ermissions and limitations under the License.
```

# Understand Better What Happens When You Run a Langchain Chain

<table align="left">
<td style="text-align: center">
<a href="https://colab.research.google.com/github/GoogleCloudPlatform/gcp-genai-samples/blob/main/assets/langchain_observability_snippet/langchain-observability-snippet.ipynb">
<img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
</a>
</td>
<td style="text-align: center">
<a href="https://github.com/GoogleCloudPlatform/gcp-genai-samples/blob/main/assets/langchain_observability_snippet/langchain-observability-snippet.ipynb">
<img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
</a>
</td>
<td style="text-align: center">
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/gcp-genai-samples/blob/main/assets/langchain_observability_snippet/langchain-observability-snippet.ipynb">
<img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
</a>
</td>
</table>



| | |
|----------|-------------|
| Author(s)   | Michael W. Sherman |
| Last updated | 2023 10 17: Cleanup.
| | 2023 07 30: Initial version. |



Last tested on Langchain version 0.0.316.

[Langchain](https://www.langchain.com/) is a popular framework for building LLM-based systems. However, some Langchain execution details are not surfaced in Langchain's [verbose mode](https://python.langchain.com/docs/modules/chains/how_to/debugging), which can complicate debugging and understanding chains.


The code snippet below implements a Langchain [callbacks](https://python.langchain.com/docs/modules/callbacks/) class called `AllChainDetails`, which prints out details of what's happening in each step of a chain and has optional debugging breakpoints.

`AllChainDetails` is primarily for educational use.

#### Notebook Structure

* Part 1 is the `AllChainDetails` code snippet with some instructions for use and a basic example.
* Part 2 is a more complete walkthrough for users of `AllChainDetails`.

Make sure to run part 1 before running part 2.

This notebook was tested in Colab.

## 1 - `AllChainDetails` Code Snippet and Usage

### Code

#### Install Dependencies

Install the dependencies, and make sure to restart the runtime after installation completes.

In [1]:
!pip install --user langchain==0.0.316 google-cloud-aiplatform==1.35.0 prettyprinter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


**MAKE SURE TO RESTART YOUR RUNTIME BEFORE GOING FURTHER**


#### The Code Snippet
The code in the next three cells is what you need to copy to use `AllChainDetails` elsewhere.

In [1]:
# Import dependencies.
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import AgentAction, AgentFinish, Document, LLMResult
from prettyprinter import cpprint
from typing import Any, Dict, List, Optional, Sequence, Type, Union
from uuid import UUID

In [3]:
# Two helper classes for pretty output.
class Color():
  """For easier understanding and faster manipulation of printed colors."""
  PURPLE = "\033[95m"
  CYAN = "\033[96m"
  DARKCYAN = "\033[36m"
  BLUE = "\033[94m"
  GREEN = "\033[92m"
  YELLOW = "\033[93m"
  RED = "\033[91m"
  BOLD = "\033[1m"
  UNDERLINE = "\033[4m"
  ITALICS = "\x1B[3m"
  END = "\033[0m\x1B[0m"


class OutputFormatter:
  """ Helper class to control the format of printed output from the callbacks.

  If used in prod, consider reimplementing in a way that removes hardcoding
    of where the output is written. Maybe use Python logging and then pass a
    custom configuration?
  """

  def heading(text: str) -> None:
    print(f"{Color.BOLD}{text}{Color.END}")

  def key_info(text: str) -> None:
    print(f"{Color.BOLD}{Color.DARKCYAN}{text}{Color.END}")

  def key_info_labeled(label: str,
                       contents: str,
                       contents_newlined: Optional[bool] = False
                       ) -> None:
    print(f"{Color.BOLD}{Color.DARKCYAN}{label}: {Color.END}{Color.DARKCYAN}",
          end="")
    if contents_newlined:
      contents = contents.splitlines()
    cpprint(f"{contents}")
    print(f"{Color.END}", end="")

  def debug_info(text: str) -> None:
    print(f"{Color.BLUE}{text}{Color.END}")

  def debug_info_labeled(label: str,
                         contents: str,
                         contents_newlined: Optional[bool] = False
                         ) -> None:
    print(f"{Color.BOLD}{Color.BLUE}{label}: {Color.END}{Color.BLUE}",
          end="")
    if contents_newlined:
      contents = contents.splitlines()
    cpprint(f"{contents}")
    print(f"{Color.END}", end="")

  def llm_call(text: str) -> None:
    print(f"{Color.ITALICS}{text}{Color.END}")

  def llm_output(text: str) -> None:
    print(f"{Color.UNDERLINE}{text}{Color.END}")

  def tool_call(text: str) -> None:
    print(f"{Color.ITALICS}{Color.PURPLE}{text}{Color.END}")

  def tool_output(text: str) -> None:
    print(f"{Color.UNDERLINE}{Color.PURPLE}{text}{Color.END}")

  def debug_error(text: str) -> None:
    print(f"{Color.BOLD}{Color.RED}{text}{Color.END}")

In [4]:
# Actual Langchain callback handler, this produces status updates during a
#   Langchain execution.
class AllChainDetails(BaseCallbackHandler):
  """Outputs details of chain progress and state.

  Exposes details available at callback time to each executed step in a chain.

  Method arguments in this class are based on the (most of?) the arguments
    available to the callback method, though not all implementations in this
    class use all the arguments.

  Usage:
    Pass as an argument to a langchain method or class that accepts a callback
      handler. Note that  not all langchain classes will invoke all callbacks
      when the callback handler is provided at initialization time, so the
      recommended usage is to provide the callback handler when executing a
      chain.

  Example:
    from langchain import LLMChain, PromptTemplate
    from langchain.llms import VertexAI
    import vertexai  # Comes from google-cloud-aiplatform package.
    vertexai.init(project=PROJECT_ID, location=REGION)

    llm = VertexAI(temperature=0)  # Use any LLM.
    prompt_template = "What food pairs well with {food}?"
    handler = AllChainDetails()
    llm_chain = LLMChain(
      llm=llm,
      prompt=PromptTemplate.from_template(prompt_template))
    llm_chain("chocolate", callbacks=[handler])

  Args:
    debug_mode: If True, prints more details of each chain step and activates
      breakpoints (using pdb) when unexpected behavior is detected. Note that
      the breakpoints are in the callbacks, which limits the amount of
      inspectable langchain state to what langchain surfaces to callbacks.
    out: Class for managing output, only tested with the OutputFormatter
      accompanying this class.
  """
  def __init__(self,
               debug_mode: Optional[bool] = False,
               out: Type[OutputFormatter] = OutputFormatter,
               ) -> None:
    self.debug_mode = debug_mode
    self.out = out

  def on_text(self,
              text: str,
              color: Optional[str] = None,
              end: str = "",
              **kwargs: Any,) -> None:
    """Run usually (not always) when langchain creates text for an LLM call.

    This callback is only used when debug_mode == True, since it can be
      confusing to see the blocks of text that come from this callback on top
      of the text sent to the LLM--it's much easier to understand what's going
      on by only looking at text sent to an LLM.

    """
    if self.debug_mode:
      self.out.heading(f"\n\n> Preparing text.")
      self.out.debug_info_labeled(f"Chain ID", f"{kwargs['run_id']}")
      self.out.debug_info_labeled("Parent chain ID",
                                  f"{kwargs['parent_run_id']}")
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      print(text)  # Langchain already agressively formats this.

  def on_llm_start(self,
                   serialized: Dict[str, Any],
                   prompts: List[str],
                   **kwargs: Any) -> None:
    """Run when langchain calls an LLM."""
    self.out.heading(f"\n\n> Sending text to the LLM.")
    self.out.key_info_labeled(f"Chain ID", f"{kwargs['run_id']}")
    self.out.key_info_labeled("Parent chain ID", f"{kwargs['parent_run_id']}")

    if len(prompts) > 1:
      self.out.debug_error("prompts has multiple items.")
      self.out.debug_error("Only outputting first item in prompts.")
      if self.debug_mode:
        self.out.debug_info_labeled("Prompts", f"{prompts}")
        breakpoint()

    self.out.key_info(f"Text sent to LLM:")
    self.out.llm_call(prompts[0])

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("serialized", f"{serialized}")

  def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
    """Run after LLM response is received by langchain."""
    self.out.heading(f"\n\n> Received response from LLM.")
    self.out.key_info_labeled(f"Chain ID", f"{kwargs['run_id']}")
    self.out.key_info_labeled("Parent chain ID", f"{kwargs['parent_run_id']}")

    if len(response.generations) > 1:
      self.out.debug_error("response object has multiple generations.")
      self.out.debug_error("Only outputting first generation in response.")
      if self.debug_mode:
        self.out.debug_info_labeled("response", f"{response}")
        breakpoint()

    self.out.key_info(f"Text received from LLM:")
    self.out.llm_output(response.generations[0][0].text)

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("response", f"{response}")

  def on_chain_start(self,
                     serialized: Dict[str, Any],
                     inputs: Dict[str, Any],
                     **kwargs: Any) -> None:
    """Run when a new chain (or subchain) is started."""
    self.out.heading(f"\n\n> Starting new chain.")

    if 'id' not in serialized.keys():
      self.out.debug_error("Missing serialized['id']")
      class_name = "Unknown -- serialized['id'] is missing"
      if self.debug_mode:
        self.out.debug_info_labeled("serialized", f"{serialized}")
        breakpoint()
    else:
      class_name = ".".join(serialized['id'])

    self.out.key_info_labeled(f"Chain class", f"{class_name}")
    self.out.key_info_labeled(f"Chain ID", f"{kwargs['run_id']}")
    self.out.key_info_labeled("Parent chain ID", f"{kwargs['parent_run_id']}")

    if len(inputs) < 1:
      self.out.debug.error("Chain inputs is empty.")
      if self.debug_mode:
        self.out.debug_info_labeled("inputs", f"{inputs}")
        breakpoint()
    else:
      self.out.key_info("Iterating through keys/values of chain inputs:")
    for key, value in inputs.items():
      # These keys contain mostly noise.
      if key not in ["stop", "agent_scratchpad"]:
        self.out.key_info_labeled(f"   {key}", f"{value}")

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("inputs", f"{inputs}")
      self.out.debug_info_labeled("serialized", f"{serialized}")

  def on_chain_end(self, outputs: Dict[str, Any], **kwargs: Any) -> None:
    """Run when a chain completes."""
    self.out.heading(f"\n\n> Ending chain.")
    self.out.key_info_labeled(f"Chain ID", f"{kwargs['run_id']}")
    self.out.key_info_labeled("Parent chain ID", f"{kwargs['parent_run_id']}")

    if len(outputs) == 0:
      self.out.debug_errors("No chain outputs.")
      if self.debug_mode:
        self.out.debug_info_labeled("outputs", f"{outputs}")
        breakpoint()
    else:
      outputs_keys = [*outputs.keys()]
    for key in outputs_keys:
      self.out.key_info_labeled(f"Output {key}",
                                f"{outputs[key]}",
                                contents_newlined=True)

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("outputs", f"{outputs}")

  def on_tool_start(self,
                    serialized: Dict[str, Any],
                    input_str: str,
                    **kwargs: Any,) -> None:
    """Run when making a call to a tool."""
    self.out.heading(f"\n\n> Using tool.")
    self.out.key_info_labeled(f"Chain ID", f"{kwargs['run_id']}")
    self.out.key_info_labeled("Parent chain ID", f"{kwargs['parent_run_id']}")
    self.out.key_info_labeled(f"Tool name", f"{serialized['name']}")
    self.out.key_info(f"Query sent to tool:")
    self.out.tool_call(input_str)

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("serialized", f"{serialized}")

  def on_tool_end(
      self,
      output: str,
      color: Optional[str] = None,
      observation_prefix: Optional[str] = None,
      llm_prefix: Optional[str] = None,
      **kwargs: Any,) -> None:
    """Run on response from a tool."""
    self.out.heading(f"\n\n> Received tool output.")
    self.out.key_info_labeled(f"Chain ID", f"{kwargs['run_id']}")
    self.out.key_info_labeled("Parent chain ID", f"{kwargs['parent_run_id']}")
    self.out.key_info_labeled(f"Tool name", f"{kwargs['name']}")

    if "output" not in locals():
      self.out.debug_error("No tool output.")
      if self.debug_mode:
        breakpoint()
    else:
      self.out.key_info("Response from tool:")
      self.out.tool_output(f"{output}")

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("observation_prefix",
                                  f"{observation_prefix}")
      self.out.debug_info_labeled("llm_prefix",
                                  f"{llm_prefix}")

  def on_agent_action(self,
                      action: AgentAction,
                      color: Optional[str] = None,
                      **kwargs: Any) -> Any:
    """Run when agent performs an action."""
    self.out.heading(f"\n\n> Agent taking an action.")
    self.out.key_info_labeled(f"Chain ID", f"{kwargs['run_id']}")
    self.out.key_info_labeled("Parent chain ID", f"{kwargs['parent_run_id']}")

    if not hasattr(action, "log"):
      self.out.debug_error("No log in action.")
      if self.debug_mode:
        self.out.debug_info_labeled("action", f"{action}")
        breakpoint()
    else:
      self.out.key_info_labeled(f"Action log",
                                f"{action.log}",
                                contents_newlined=True)

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("action", f"{action}")

  def on_agent_finish(self,
                      finish: AgentFinish,
                      color: Optional[str] = None,
                      **kwargs: Any) -> None:
    """Run after agent completes."""
    self.out.heading(f"\n\n> Agent has finished.")
    self.out.key_info_labeled(f"Chain ID", f"{kwargs['run_id']}")
    self.out.key_info_labeled("Parent chain ID", f"{kwargs['parent_run_id']}")

    if not hasattr(finish, "log"):
      self.out.debug_error("No log in action finish.")
      if self.debug_mode:
        breakpoint()
    else:
      self.out.key_info_labeled(f"Action finish log",
                                f"{finish.log}",
                                contents_newlined=True)

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("finish",
                                  f"{finish}")

  def on_llm_error(self,
                   error: Union[Exception, KeyboardInterrupt],
                   **kwargs: Any) -> None:
    self.out.debug_error("LLM Error")
    self.out.debug_info_labeled("Error object", f"{error}")
    if self.debug_mode:
      breakpoint()

  def on_chain_error(self,
                     error: Union[Exception, KeyboardInterrupt],
                     **kwargs: Any) -> None:
    self.out.debug_error("Chain Error")
    self.out.debug_info_labeled("Error object", f"{error}")
    if self.debug_mode:
      breakpoint()

  def on_tool_error(self,
                    error: Union[Exception, KeyboardInterrupt],
                    **kwargs: Any) -> None:
    self.out.debug_error("Chain Error")
    self.out.debug_info_labeled("Error object", f"{error}")
    if self.debug_mode:
      breakpoint()

  def on_retriever_start(self,
                         serialized: Dict[str, Any],
                         query: str,
                         *,
                         run_id: UUID,
                         parent_run_id: Optional[UUID] = None,
                         tags: Optional[List[str]] = None,
                         metadata: Optional[Dict[str, Any]] = None,
                         **kwargs: Any) -> Any:
    """Run when querying a retriever."""
    self.out.heading(f"\n\n> Querying retriever.")
    self.out.key_info_labeled(f"Chain ID", f"{run_id}")
    self.out.key_info_labeled("Parent chain ID", f"{parent_run_id}")
    self.out.key_info_labeled("Tags", f"{tags}")

    if 'id' not in serialized.keys():
      self.out.debug_error("Missing serialized['id']")
      class_name = "Unknown -- serialized['id'] is missing"
      if self.debug_mode:
        self.out.debug_info_labeled("serialized", f"{serialized}")
        breakpoint()
    else:
      class_name = ".".join(serialized['id'])
    self.out.key_info_labeled(f"Retriever class", f"{class_name}")

    self.out.key_info(f"Query sent to retriever:")
    self.out.tool_call(query)

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("metadata", f"{metadata}")
      self.out.debug_info_labeled("serialized", f"{serialized}")

  def on_retriever_end(self,
                       documents: Sequence[Document],
                       *,
                       run_id: UUID,
                       parent_run_id: Optional[UUID] = None,
                       **kwargs: Any) -> Any:
    """Run when retriever returns a response."""
    self.out.heading(f"\n\n> Retriever finished.")
    self.out.key_info_labeled(f"Chain ID", f"{run_id}")
    self.out.key_info_labeled("Parent chain ID", f"{parent_run_id}")
    self.out.key_info(f"Found {len(documents)} documents.")

    if len(documents) == 0:
      self.out.debug_error("No documents found.")
      if self.debug_mode:
        breakpoint()
    else:
      for doc_num, doc in enumerate(documents):
        self.out.key_info("---------------------------------------------------")
        self.out.key_info(f"Document number {doc_num} of {len(documents)}")
        self.out.key_info_labeled("Metadata", f"{doc.metadata}")
        self.out.key_info("Document contents:")
        self.out.tool_output(doc.page_content)

### `AllChainDetails` Usage

If you're using Colab, run the code in the next cell. Follow the popups and authenticate with an account that has access to a Google Cloud [project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects), for using the [Vertex AI LLMs](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

If you're running this notebook somewhere besides Colab, make sure your environment has the right Google Cloud access. If that's a new concept to you, consider looking into [Application Default Credentials for your local environment](https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev). More authentication options are discussed [here](https://cloud.google.com/docs/authentication).

If you're entirely new to Google Cloud, [get started](https://cloud.google.com/docs/get-started).

In [5]:
# Colab authentication.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
    print('Authenticated')

In [6]:
PROJECT_ID = "YOUR_PROJECT_ID_HERE"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
# Code examples may misbehave if the model is changed.
MODEL_NAME = "text-bison@001"

In [7]:
# Dependencies for usage example.
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import VertexAI
import vertexai  # Comes from google-cloud-aiplatform package.

# Initiaize connection to Vertex PaLM API LLM.
vertexai.init(project=PROJECT_ID, location=LOCATION)
llm = VertexAI(model_name=MODEL_NAME, temperature=0)

You can use the `AllChainDetails` callback handler both when executing a chain/agent/etc. or when initializing a chain/agent/etc.

You'll generally get more complete output of langchain internals when passing the `AllChainDetails` callback handler to a chain execution rather than an initialization.

In [8]:
# Callback handler specified at execution time, more information given.
prompt_template = "What food pairs well with {food}?"
handler = AllChainDetails()
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("chocolate", callbacks=[handler])



> Starting new chain.
Chain class: 'langchain.chains.llm.LLMChain'
Chain ID: 'd2b274f7-b992-4b67-a352-8968bd9efa1f'
Parent chain ID: 'None'
Iterating through keys/values of chain inputs:
   food: 'chocolate'


> Sending text to the LLM.
Chain ID: 'bda6c996-7750-48fc-ba05-7afe5c18933b'
Parent chain ID: 'd2b274f7-b992-4b67-a352-8968bd9efa1f'
Text sent to LLM:
What food pairs well with chocolate?


> Received response from LLM.
Chain ID: 'bda6c996-7750-48fc-ba05-7afe5c18933b'
Parent chain ID: 'd2b274f7-b992-4b67-a352-8968bd9efa1f'
Text received from LLM:
Chocolate pairs well with many foods, including fruits, nuts, and dairy products. Some popular pairings include chocolate with strawberries, chocolate with bananas, chocolate with nuts, and chocolate with cheese.


> Ending chain.
Chain ID: 'd2b274f7-b992-4b67-a352-8968bd9efa1f'
Parent chain ID: 'None'
Output text: "['Chocolate pairs well with many foods, including fruits, nuts, and "
"dairy products. Some popular pairings include choco

{'food': 'chocolate',
 'text': 'Chocolate pairs well with many foods, including fruits, nuts, and dairy products. Some popular pairings include chocolate with strawberries, chocolate with bananas, chocolate with nuts, and chocolate with cheese.'}

In [9]:
# Callback handler specified at initialization, less information given.
prompt_template = "What food pairs well with {food}?"
handler = AllChainDetails()
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template),
    callbacks=[handler])
llm_chain("chocolate")



> Starting new chain.
Chain class: 'langchain.chains.llm.LLMChain'
Chain ID: '9b321f38-174a-4258-99a8-25c611585553'
Parent chain ID: 'None'
Iterating through keys/values of chain inputs:
   food: 'chocolate'


> Ending chain.
Chain ID: '9b321f38-174a-4258-99a8-25c611585553'
Parent chain ID: 'None'
Output text: "['Chocolate pairs well with many foods, including fruits, nuts, and "
"dairy products. Some popular pairings include chocolate with "
"strawberries, chocolate with bananas, chocolate with nuts, and "
"chocolate with cheese.']"


{'food': 'chocolate',
 'text': 'Chocolate pairs well with many foods, including fruits, nuts, and dairy products. Some popular pairings include chocolate with strawberries, chocolate with bananas, chocolate with nuts, and chocolate with cheese.'}

##### Debug Mode

`AllChainDetails` has a debug mode that provides more output information and engages breakpoints when a chain errors or something unexpected happens.

In [10]:
prompt_template = "What food pairs well with {food}?"
# Turn on debug mode.
handler = AllChainDetails(debug_mode=True)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("chocolate", callbacks=[handler])



> Starting new chain.
Chain class: 'langchain.chains.llm.LLMChain'
Chain ID: '941c6dd8-1474-465f-a34b-7a31ac30c04f'
Parent chain ID: 'None'
Iterating through keys/values of chain inputs:
   food: 'chocolate'
Arguments: "{'run_id': UUID('941c6dd8-1474-465f-a34b-7a31ac30c04f'), "
"'parent_run_id': None, 'tags': [], 'metadata': {}, 'name': None}"
inputs: "{'food': 'chocolate'}"
serialized: "{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'chains', "
"'llm', 'LLMChain'], 'kwargs': {'llm': {'lc': 1, 'type': "
"'constructor', 'id': ['langchain', 'llms', 'vertexai', 'VertexAI'], "
"'kwargs': {'model_name': 'text-bison@001', 'temperature': 0.0}}, "
"'prompt': {'lc': 1, 'type': 'constructor', 'id': ['langchain', "
"'prompts', 'prompt', 'PromptTemplate'], 'kwargs': "
"{'input_variables': ['food'], 'template': 'What food pairs well with "
"{food}?', 'template_format': 'f-string', 'partial_variables': {}}}}}"


> Preparing text.
Chain ID: '941c6dd8-1474-465f-a34b-7a31ac30c04f'
Parent chain 

{'food': 'chocolate',
 'text': 'Chocolate pairs well with many foods, including fruits, nuts, and dairy products. Some popular pairings include chocolate with strawberries, chocolate with bananas, chocolate with nuts, and chocolate with cheese.'}

**Tip**: New to [Python debugging](https://docs.python.org/3/library/pdb.html#debugger-commands)? Just type 'c' then enter in the text box that appears at the bottom of the cell's output when the execution breaks.

In [11]:
# Temperature > 1 causes the PaLM APIs to return an error.
llm = VertexAI(model_name=MODEL_NAME, temperature=10)
prompt_template = "What food pairs well with {food}?"
handler = AllChainDetails(debug_mode=True)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("testing", callbacks=[handler])



> Starting new chain.
Chain class: 'langchain.chains.llm.LLMChain'
Chain ID: 'fd0bb489-4a20-47ae-a542-db2e4a3eb508'
Parent chain ID: 'None'
Iterating through keys/values of chain inputs:
   food: 'testing'
Arguments: "{'run_id': UUID('fd0bb489-4a20-47ae-a542-db2e4a3eb508'), "
"'parent_run_id': None, 'tags': [], 'metadata': {}, 'name': None}"
inputs: "{'food': 'testing'}"
serialized: "{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'chains', "
"'llm', 'LLMChain'], 'kwargs': {'llm': {'lc': 1, 'type': "
"'constructor', 'id': ['langchain', 'llms', 'vertexai', 'VertexAI'], "
"'kwargs': {'model_name': 'text-bison@001', 'temperature': 10.0}}, "
"'prompt': {'lc': 1, 'type': 'constructor', 'id': ['langchain', "
"'prompts', 'prompt', 'PromptTemplate'], 'kwargs': "
"{'input_variables': ['food'], 'template': 'What food pairs well with "
"{food}?', 'template_format': 'f-string', 'partial_variables': {}}}}}"


> Preparing text.
Chain ID: 'fd0bb489-4a20-47ae-a542-db2e4a3eb508'
Parent chain ID:


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 336, in set_trace
    sys.settrace(self.trace_dispatch)



LLM Error
Error object: '400 10.000000 is out of supported range [0, 1];  for value of '
'temperature.'
--Return--
None
> <ipython-input-4-aee70565b176>(267)on_llm_error()
    265     self.out.debug_info_labeled("Error object", f"{error}")
    266     if self.debug_mode:
--> 267       breakpoint()
    268 
    269   def on_chain_error(self,

ipdb> c



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 347, in set_continue
    sys.settrace(None)



Chain Error
Error object: '400 10.000000 is out of supported range [0, 1];  for value of '
'temperature.'
--Return--
None
> <ipython-input-4-aee70565b176>(275)on_chain_error()
    273     self.out.debug_info_labeled("Error object", f"{error}")
    274     if self.debug_mode:
--> 275       breakpoint()
    276 
    277   def on_tool_error(self,

ipdb> c


InvalidArgument: ignored

# 2 - Introductory Walkthrough

`AllChainDetails` is most useful with Langchain agents, since some details are not available during chain execution.

We'll create an ReAct agent that has access to Wikpedia search and calculator tools, then observe the steps that happen during agent execution.

In [12]:
# One more dependency, no need to restart.
!pip install --user wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=8c952630d294232439cf6da1235377b20a4647dc345a3308fff5af7836c887f2
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [13]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
import wikipedia

llm = VertexAI(model_name=MODEL_NAME, temperature=0)
# Initialize the Wikipedia tool.
_ = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
# This next line invisibly maps to the previous line. The WikipediaQueryRun
#   call is what matters here for Langchain to use its "wikipedia", not
#   the variable that call is output to.

tools = load_tools(["wikipedia", "llm-math"], llm=llm)
handler = AllChainDetails()

In [14]:
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)
agent.run("What US President costarred with a chimp in 'Bedtime for Bonzo'?",
          callbacks=[handler])



> Starting new chain.
Chain class: 'langchain.agents.agent.AgentExecutor'
Chain ID: '38181f4e-4bbc-4ee4-811f-31ccebba88aa'
Parent chain ID: 'None'
Iterating through keys/values of chain inputs:
   input: "What US President costarred with a chimp in 'Bedtime for Bonzo'?"


> Starting new chain.
Chain class: 'langchain.chains.llm.LLMChain'
Chain ID: '39282b60-b054-4f88-9e7f-22a95472f17f'
Parent chain ID: '38181f4e-4bbc-4ee4-811f-31ccebba88aa'
Iterating through keys/values of chain inputs:
   input: "What US President costarred with a chimp in 'Bedtime for Bonzo'?"


> Sending text to the LLM.
Chain ID: '25d07543-4963-4014-a637-1ae78ed76171'
Parent chain ID: '39282b60-b054-4f88-9e7f-22a95472f17f'
Text sent to LLM:
Answer the following questions as best you can. You have access to the following tools:

Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a 

/root/.local/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /root/.local/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')




> Received tool output.
Chain ID: 'ae13987c-d276-4610-b225-e3c85810e607'
Parent chain ID: '38181f4e-4bbc-4ee4-811f-31ccebba88aa'
Tool name: 'Wikipedia'
Response from tool:
Page: Bedtime for Bonzo
Summary: Bedtime for Bonzo is a 1951 American comedy film directed by Fred de Cordova and starring Ronald Reagan, Diana Lynn, and a chimpanzee named Peggy as Bonzo. Its central character, psychology professor Peter Boyd (Reagan), tries to teach human morals to a chimpanzee, hoping to solve the "nature versus nurture" question. Boyd hires Jane Linden (Lynn) to pose as the chimpanzee's mother while he plays father to it and uses 1950s-era child-rearing techniques.A sequel was released titled Bonzo Goes to College (1952), but it featured none of the three lead performers from the original film. Peggy, who had also appeared in My Friend Irma Goes West (1950), died in a fire on March 4, 1951, so another chimpanzee was hired for the second film. Reagan did not want to appear in the second film as 

'Ronald Reagan'

It's now possible to follow the complete set of calls to the LLM and track all the calls out to Wikipedia. Compare this to Langchain's built-in verbose mode, which doesn't print the complete LLM prompts.

In [15]:
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)
agent.run("What US President costarred with a chimp in 'Bedtime for Bonzo'?")



> Entering new AgentExecutor chain...
I need to find out what US President costarred with a chimp in 'Bedtime for Bonzo'
Action: Wikipedia
Action Input: bedtime for bonzo

/root/.local/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /root/.local/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Bedtime for Bonzo
Summary: Bedtime for Bonzo is a 1951 American comedy film directed by Fred de Cordova and starring Ronald Reagan, Diana Lynn, and a chimpanzee named Peggy as Bonzo. Its central character, psychology professor Peter Boyd (Reagan), tries to teach human morals to a chimpanzee, hoping to solve the "nature versus nurture" question. Boyd hires Jane Linden (Lynn) to pose as the chimpanzee's mother while he plays father to it and uses 1950s-era child-rearing techniques.A sequel was released titled Bonzo Goes to College (1952), but it featured none of the three lead performers from the original film. Peggy, who had also appeared in My Friend Irma Goes West (1950), died in a fire on March 4, 1951, so another chimpanzee was hired for the second film. Reagan did not want to appear in the second film as he thought that the premise was unbelievable.

Page: Bedtime for Democracy
Summary: Bedtime for Democracy is the fourth and final studio album by American punk 

'Ronald Reagan'

Verbose mode can also hide important details. Can you tell the cause of this failure?

In [17]:
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)
agent.run("What day of the week was September 1st, 2010?")



> Entering new AgentExecutor chain...
I need to know what day of the week September 1st, 2010 was
Action: Calculator
Action Input: 1 September 2010

ValueError: ignored

When you can see the full call langchain makes to the LLM to use the math tool, it's easier to see that the failure is due to the LLM returning a response to the math tool prompt that can't be parsed by `numexpr`:

In [18]:
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=False)
agent.run("What day of the week was September 1st, 2010?",
          callbacks=[handler])



> Starting new chain.
Chain class: 'langchain.agents.agent.AgentExecutor'
Chain ID: '7aa29d2a-f2d3-49bb-ba4e-934115d996a8'
Parent chain ID: 'None'
Iterating through keys/values of chain inputs:
   input: 'What day of the week was September 1st, 2010?'


> Starting new chain.
Chain class: 'langchain.chains.llm.LLMChain'
Chain ID: '23739bfa-fd3b-40b2-a499-64a06d9e7959'
Parent chain ID: '7aa29d2a-f2d3-49bb-ba4e-934115d996a8'
Iterating through keys/values of chain inputs:
   input: 'What day of the week was September 1st, 2010?'


> Sending text to the LLM.
Chain ID: '7cb565f7-481f-40c1-9b2c-d1562aa71aac'
Parent chain ID: '23739bfa-fd3b-40b2-a499-64a06d9e7959'
Text sent to LLM:
Answer the following questions as best you can. You have access to the following tools:

Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Calculator: Useful for w

ValueError: ignored

It can be difficult to see how different built-in Langchain agents types prompt the LLM differently, even with `verbose=True`.

The first example here uses the same agent setup as above, with a Wikipedia and math tool:

In [23]:
question = "What TV show inspired the saying 'Jumping the Shark'?"
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)
agent.run(question)



> Entering new AgentExecutor chain...
I need to know what TV show inspired the saying 'Jumping the Shark'
Action: Wikipedia
Action Input: jumping the shark
Observation: Page: Jumping the shark
Summary: The idiom "jumping the shark" or "jump the shark" is a pejorative that is used to argue that a creative work or entity has reached a point in which it has exhausted its core intent and is introducing new ideas that are discordant with, or an overexaggeration of, its original purpose. The phrase was coined in 1985 by radio personality Jon Hein in response to a 1977 episode from the fifth season of the American sitcom Happy Days, in which the character of Fonzie (Henry Winkler) jumps over a live shark while on water-skis.



Page: Jump the Shark (The X-Files)
Summary: "Jump the Shark" is the fifteenth episode of the ninth season of the American science fiction television series The X-Files. The episode first aired in the United States on April 21, 2002 on the Fox network. It was written 

'Happy Days'

The same query, sent to a docstore agent:

In [24]:
from langchain.agents import Tool
from langchain.agents.react.base import DocstoreExplorer
from langchain import Wikipedia

docstore = DocstoreExplorer(Wikipedia())
doc_tools = [
    Tool(name="Search",
         func=docstore.search,
         description="useful for when you need to ask with search",),
    Tool(name="Lookup",
         func=docstore.lookup,
         description="useful for when you need to ask with lookup",),
]
doc_agent = initialize_agent(doc_tools,
                             llm,
                             agent=AgentType.REACT_DOCSTORE,
                             verbose=True)
doc_agent({"input": question})



> Entering new AgentExecutor chain...
Thought: I need to search "Jumping the Shark" and find the TV show that inspired the saying.
Action: Search[Jumping the Shark]
Observation: The idiom "jumping the shark" or "jump the shark" is a pejorative that is used to argue that a creative work or entity has reached a point in which it has exhausted its core intent and is introducing new ideas that are discordant with, or an overexaggeration of, its original purpose. The phrase was coined in 1985 by radio personality Jon Hein in response to a 1977 episode from the fifth season of the American sitcom Happy Days, in which the character of Fonzie (Henry Winkler) jumps over a live shark while on water-skis.
Thought:The idiom "jumping the shark" was coined in 1985 by radio personality Jon Hein in response to a 1977 episode from the fifth season of the American sitcom Happy Days, in which the character of Fonzie (Henry Winkler) jumps over a live shark while on water-skis. So the TV show that inspir

{'input': "What TV show inspired the saying 'Jumping the Shark'?",
 'output': 'Happy Days'}

Running these with the `AllChainDetails` callback handler more clearly shows how the agents prompt the LLM differently.

In [25]:
agent.run(question, callbacks=[handler])



> Starting new chain.
Chain class: 'langchain.agents.agent.AgentExecutor'
Chain ID: '11f5945c-1cb0-4dae-87b7-5faee4d7f554'
Parent chain ID: 'None'
Iterating through keys/values of chain inputs:
   input: "What TV show inspired the saying 'Jumping the Shark'?"


> Entering new AgentExecutor chain...


> Starting new chain.
Chain class: 'langchain.chains.llm.LLMChain'
Chain ID: 'e186a09b-6d92-4ff8-ae38-53fb0fdcdcbd'
Parent chain ID: '11f5945c-1cb0-4dae-87b7-5faee4d7f554'
Iterating through keys/values of chain inputs:
   input: "What TV show inspired the saying 'Jumping the Shark'?"


> Sending text to the LLM.
Chain ID: 'ce757510-a537-446f-9c53-b492e273d406'
Parent chain ID: 'e186a09b-6d92-4ff8-ae38-53fb0fdcdcbd'
Text sent to LLM:
Answer the following questions as best you can. You have access to the following tools:

Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. 

'Happy Days'

In [26]:
doc_agent({"input": question}, callbacks=[handler])



> Starting new chain.
Chain class: 'langchain.agents.agent.AgentExecutor'
Chain ID: 'faa0c084-047f-4b9e-8cfe-7dedff07ee6e'
Parent chain ID: 'None'
Iterating through keys/values of chain inputs:
   input: "What TV show inspired the saying 'Jumping the Shark'?"


> Entering new AgentExecutor chain...


> Starting new chain.
Chain class: 'langchain.chains.llm.LLMChain'
Chain ID: '99d20ccf-685c-49db-8264-4a5a29ca8281'
Parent chain ID: 'faa0c084-047f-4b9e-8cfe-7dedff07ee6e'
Iterating through keys/values of chain inputs:
   input: "What TV show inspired the saying 'Jumping the Shark'?"


> Sending text to the LLM.
Chain ID: 'a6bfe585-eadc-41ab-9259-0e3c172aac43'
Parent chain ID: '99d20ccf-685c-49db-8264-4a5a29ca8281'
Text sent to LLM:


Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation 

{'input': "What TV show inspired the saying 'Jumping the Shark'?",
 'output': 'Happy Days'}